## Code is copied from: https://scikit-gstat.readthedocs.io/en/latest/userguide/kriging.html
### Functions as a first idea, to maybe implement this process in Python

In [17]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
from skgstat.models import spherical
from scipy.linalg import solve


In [ ]:
Z_s = np.array([4.2, 6.1, 0.2, 0.7, 5.2])

lam = np.array([0.1, 0.3, 0.1, 0.1, 0.4])

# calculate the weighted mean
np.sum(Z_s * lam) # or Z_s.dot(lam)

4.42

In [9]:
x = np.array([4.0, 2.0, 4.1, 0.3, 2.0])

y = np.array([5.5, 1.2, 3.7, 2.0, 2.5])

z = np.array([4.2, 6.1, 0.2, 0.7, 5.2])

s0 = [2., 2.]

distance_matrix = pdist([s0] + list(zip(x,y)))

squareform(distance_matrix)

array([[0.        , 4.03112887, 0.8       , 2.70185122, 1.7       ,
        0.5       ],
       [4.03112887, 0.        , 4.74236228, 1.80277564, 5.09313263,
        3.60555128],
       [0.8       , 4.74236228, 0.        , 3.26496554, 1.87882942,
        1.3       ],
       [2.70185122, 1.80277564, 3.26496554, 0.        , 4.16293166,
        2.41867732],
       [1.7       , 5.09313263, 1.87882942, 4.16293166, 0.        ,
        1.77200451],
       [0.5       , 3.60555128, 1.3       , 2.41867732, 1.77200451,
        0.        ]])

In [13]:
# range= 7. sill = 2. nugget = 0.
model = lambda h: spherical(h, 7.0, 2.0, 0.0)

variances = model(distance_matrix[:5])

assert len(variances) == 5

In [ ]:
dists = pdist(list(zip(x,y)))

M = squareform(model(dists))

print(M)

print(variances)

[1.53664752 0.34136443 1.10043328 0.71424781 0.21392128]


In [18]:
# solve for a
a = solve(M, variances)

print(a)

# calculate estimation
Z_s.dot(a)

[-0.02155993  0.36166541  0.01802147  0.03655808  0.59258941]


5.226267185422778

In [19]:
np.sum(a)

0.9872744357166218

In [ ]:
B = np.concatenate((variances, [1]))

M = np.concatenate((M, [[1, 1, 1, 1, 1]]), axis=0)

M = np.concatenate((M, [[1], [1], [1], [1], [1], [0]]), axis=1)

weights = solve(M, B)

# see the weights
print('Old weights:', a)

print('New weights:', weights[:-1])

print('Old estimation:', Z_s.dot(a))

print('New estimation:', Z_s.dot(weights[:-1]))

print('Mean:', np.mean(Z_s))


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 6 and the array at index 1 has size 5

In [23]:
np.sum(weights[:-1])

sum(B[:-1] * weights[:-1]) + weights[-1]

0.2628757539286831

In [ ]:
data = pd.read_csv('data/sample_lr.csv')

In [36]: V = Variogram(data[['x', 'y']].values, data.z.values,
   ....:   maxlag=90, n_lags=25, model='gaussian', normalize=False)
   ....: 

In [37]: V.plot()
Out[37]: <Figure size 800x500 with 2 Axes>

In [38]: from skgstat import OrdinaryKriging

In [39]: ok = OrdinaryKriging(V, min_points=5, max_points=20, mode='exact')